# Demonstration of the frequency response of LTI systems

This demo is supposed to visualize the impulse and frequency resoponse of a continous time linear time-invariant system. In this demo a simple RC lowpass filter is used as axample. 

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on signals and systems hold at the [university of applied sciences, Berlin](https://www.htw-berlin.de/).

**Import libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import ipywidgets as widgets

plt.style.use('noelle.mplstyle')

**Effect of a RC lowpass on a rectangular input signal**

A rectangular input signal of the form
$$ x(t)=\sqcap_{T_0}(t-t_0) $$
is passed through a [linear time-invariant (LTI) system](https://en.wikipedia.org/wiki/Linear_time-invariant_system). In this case, the system is a [first order RC lowpass filter](https://en.wikipedia.org/wiki/Low-pass_filter#RC_filter) shown below  

![RC lowpass](https://upload.wikimedia.org/wikipedia/commons/e/e0/1st_Order_Lowpass_Filter_RC.svg)

This system has an [impulse response](https://en.wikipedia.org/wiki/Impulse_response) of 
$$ h(t)=\sigma(t)\frac{1}{\tau}e^{-t/\tau},$$
where $\tau=RC$ with $R$ being the [resistance](https://en.wikipedia.org/wiki/Electrical_resistance_and_conductance) in Ohm and $C$ being the [capacitance](https://en.wikipedia.org/wiki/Capacitance) in Farad, respectively. Further, $\sigma(t)$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function). This corrsponds to a [frequency response](https://en.wikipedia.org/wiki/Frequency_response) of the form
$$ H(j\omega)= \frac{1}{1+j\omega\tau}.$$
 
Following the theory of linear systems, the output signal is given as a convolution of the input signal and the impulse response
$$y(t)= x(t)\ast h(t),$$
or equivalently, as a multiplication of the input sepectrum and the frequency response
$$Y(j\omega)= X(j\omega)\cdot H(j\omega).$$

This relationship is shown in the following animation, while the [time constant ($\tau$)](https://en.wikipedia.org/wiki/Time_constant) and therewith also the [cutoff frequency](https://en.wikipedia.org/wiki/Cutoff_frequency) of the filter can be varied.


In [2]:
def testFun(tau=0.1):
    
    sr = 100 # sample rate

    dur = 10 # duration of signals in s
    T0 = 1 # width of rectangle in s
    T = T0/2 # repetition period in s 
    dutyCyle = T0 / T # duty cycle of rect
    t0 = 1 # tempral shift of pulse
    f_cutoff = 1/tau/2/np.pi

    t = np.arange(0, dur, 1 / sr)
    f = np.fft.fftshift(np.fft.fftfreq(t.size, 1 / sr))

    # generate signal
    data = np.zeros(t.size)

    # rect-pulse
    data[(t-t0 > (T-T0/2)) & (t-t0 < (T+T0/2))] = 1

    # calc spectrum of data signal (and normalize)
    Data = np.fft.fft(data)
    DataNorm = Data / np.max(np.abs(Data))

    # exponential decaying h of length 1 s
    h = np.exp(-t/tau)/tau
    h = h / np.sum(h)

    # frequency response (and normalize)
    H = np.fft.fft(h)
    HNorm = H / np.max(np.abs(H))

    # plots
    n_row = 2
    n_col = 2
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig, ax = plt.subplots(n_row, n_col, figsize=fig_size)
   
    ax[0][0].plot(t,h/h[0])
    ax[0][0].set_xlabel('time / s')
    ax[0][0].set_ylabel('h(t)/h(0)')
    ax[0][0].set_xlim(-1, 5) 
    
    ax[0][1].plot(f, np.fft.fftshift(np.abs(HNorm)))
    ax[0][1].set_xlabel('frequency / Hz')
    ax[0][1].set_ylabel('$|H(j\omega)/H(0)|$')
    ax[0][1].set_xlim(-10, 10)
    ax[0][1].set_ylim(0, 1.1)
    ax[0][1].text(-9, 1.0, 'cutoff frequency: {0:.1f} Hz'.format(f_cutoff))
    
    
    DataFiltNorm = DataNorm * HNorm
    DataFilt = Data * H
    dataFilt = np.real(np.fft.ifft(DataFilt))

    ax[1][1].plot(f, np.fft.fftshift(np.abs(DataNorm)), f, np.fft.fftshift(np.abs(DataFiltNorm)))
    ax[1][1].set_xlabel('frequency / Hz')
    ax[1][1].set_ylabel('norm. amplitude / a.u.')
    ax[1][1].set_xlim(-10, 10)    
    ax[1][1].legend(['input signal $(|X(j\omega)|)$','output signal $(|Y(j\omega)|)$'])  

    ax[1][0].plot(t, data, t, dataFilt)
    ax[1][0].set_xlabel('time / s')
    ax[1][0].set_ylabel('amplitude / a.u.') 
    ax[1][0].set_xlim(0, 5)   
    ax[1][0].legend(['input signal (x(t))','output signal (y(t))'])    
    
    plt.tight_layout()
    plt.show()

w_tau = widgets.FloatSlider(min=0.01, max=0.8, step=0.01, value=0.01, continuous_update=False, description=r'$\tau$:')

out = widgets.interactive_output(testFun, {'tau':w_tau})
out.layout.height = '600px'

display(w_tau, out)

FloatSlider(value=0.01, continuous_update=False, description='$\\tau$:', max=0.8, min=0.01, step=0.01)

Output(layout=Layout(height='600px'))